# OpenAlex Concept Matching at scale using Azure's CosmosDB

### Once the datasets (academic, financial, patents) are preprocessed and concepts are tagged for each document, they can then be uploaded to CosmosDB (NoSQL database). Following are the steps required to upload the preprocessed documents to CosmosDB and query across financial, academic, and patent datasets -   

### Step1: Create an Azure Account, setup CosmosDB account, and create a database for this project. Download API keys(MASTER_KEY) and copy host URI(HOST) of the project from the "Keys" section of Database Settings.

### Step2: Create respective containers for each dataset using CosmosDB API.

First, install cosmosdb python API using ```pip install azure-cosmos```

In [ ]:
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey

In the example below we create a container for companies (from Refinitiv financial dataset)

In [ ]:
client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
try:
    # setup database for this sample
    try:
        db = client.create_database(id=DATABASE_ID)
        print('Database with id \'{0}\' created'.format(DATABASE_ID))

    except exceptions.CosmosResourceExistsError:
        db = client.get_database_client(DATABASE_ID)
        print('Database with id \'{0}\' was found'.format(DATABASE_ID))

    # setup container for this sample
    try:
        container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/year'))
        print('Container with id \'{0}\' created'.format(CONTAINER_ID))

    except exceptions.CosmosResourceExistsError:
        container = db.get_container_client(CONTAINER_ID)
        print('Container with id \'{0}\' was found'.format(CONTAINER_ID))
        
except exceptions.CosmosHttpResponseError as e:
    print('\nrun_sample has caught an error. {0}'.format(e.message))

finally:
    print("\nrun_sample done")

### Step 3 - Prepare Dataset and Upload to the respective container

(i) To prepare the dataset for upload, it should be loaded as a Pandas dataframe or JSON object, such that it can be uploaded to container as JSON.

In [5]:
import pandas as pd
import json

# Load dataframe using json.load() or pd.read_csv/excel()
df = pd.DataFrame()

(ii) - Ensure that there are no special characters or whitespaces in columns names, such as colons, hyphens etc. Replace all special characters with an Underscore or "".

In [4]:
for cols in list(df):
    old_col = cols
    cols = cols.replace("\n","")
    cols = cols.replace("('|')","")
    cols = cols.replace(",","")
    cols = cols.replace(".","")
    cols = cols.replace("-","")
    cols = cols.replace("(","_")
    cols = cols.replace(")","")
    cols = "_".join(cols.split(" "))
    cols = cols.lower()
    df.rename(columns={old_col:cols}, inplace=True)

(iii) Ensure that there are no NaN values in the dataframe/JSON. Replace them with None or remove them.

In [7]:
import numpy as np
df = pd.DataFrame(df).replace({np.nan:None})

(iv) Although CosmosDB creates an index across all fields by default, it still requires an "id" key during document upload (see https://learn.microsoft.com/en-us/azure/cosmos-db/nosql/troubleshoot-bad-request). Therefore, assign a unique id for all documents if it doesn't exist already.

In [8]:
import uuid

# Function to generate a unique string index
def generate_unique_index():
    return str(uuid.uuid4())

df['id'] = df.apply(lambda row: generate_unique_index(), axis=1)

(v) Ensure that concept IDs are stores as list of integers. (Generated using: [Concept Mapping](concept_mapping/concept_mapping_readme.md)). An example document now looks like - 

In [ ]:
{
    "company_name": "Arisglobal LLC",
    "company_nation": "United States",
    "investee_company_trbc_economic_sector": "Technology",
    "avg_equity_per_deal_in_search_range_usd_millions": 0,
    "avg_equity_per_firm_in_search_range_usd_millions": 0,
    "avg_equity_per_fund_in_search_range_usd_millions": 0,
    "sum_of_equity_invested_in_search_range_usd_millions": 0,
    "investee_primary_sic": "Prepackaged Software",
    "investee_company_primary_ve_industry_subgroup_2": "Internet Software",
    "investee_company_primary_veic": "Other Internet Systems Software",
    "investee_company_trbc_industry_group": "Software & IT Services",
    "investee_company_trbc_industry": "Software",
    "investee_company_trbc_business_sector": "Software & IT Services",
    "investee_company_trbc_activity": "Enterprise Software",
    "investee_company_naics_2022": "Software Publishers",
    "investee_sic": "Prepackaged Software",
    "investee_company_ve_industry_subgroup_2": "Internet Software",
    "investee_company_ve_primary_industry_subgroup_3": "Other Internet Systems Software",
    "investee_company_ve_industry_class": "Information Technology",
    "year": 2023,
    "investment_round_permid": 2,
    "firm_investor_beid": 1,
    "fund_investor_beid": 2,
    "deal_value_usd_millions": 0,
    "deal_value_usd_millions1": 0,
    "deal_rank_value_usd_millions": 0,
    "investee_company_website": "www.arisglobal.com",
    "investee_company_status": "Active",
    "investee_company_founded_date": "1987-01-01 00:00:00",
    "investee_company_long_business_description": "ArisGlobal LLC is a United States-based technology company. The Company provides cloud-based software solutions for pharmacovigilance and drug safety, clinical development, regulatory, and medical affairs. Its drug development technology platform, LifeSphere, uses its Nava cognitive computing engine to automate core functions of the drug development lifecycle. Its LifeSphere platform includes LifeSphere Clinical, LifeSphere Regulatory, LifeShpere Safety, LifeSphere Medical Affairs, LifeShpere EasyDocs and LifeSphere Cloud. The Company services include professional services, managed services and LifeSphere trainings.",
    "investee_company_short_business_description": "Provider of cloud-based software solutions for drug safety.",
    "investee_company_alias_name": None,
    "openalex_concept_ids": [
        79974875,
        57658597,
        2777904410,
        110354214,
        144133560,
        195094911,
        38652104,
        41008148
    ],
    "openalex_concept_ids_with_full_chain": [
        79974875,
        57658597,
        2777904410,
        110354214,
        144133560,
        195094911,
        38652104,
        41008148,
        71924100,
        2780035454,
        127413603,
        98274493,
        199360897,
        111919701
    ],
    "openalex_concepts": [
        "cloud computing",
        "pharmacovigilance",
        "software",
        "engineering management",
        "business",
        "process management",
        "computer security",
        "computer science"
    ],
    "openalex_concepts_with_chains": [
        "cloud computing",
        "pharmacovigilance",
        "software",
        "engineering management",
        "business",
        "process management",
        "computer security",
        "computer science",
        "medicine",
        "drug",
        "engineering",
        "pharmacology",
        "programming language",
        "operating system"
    ],
}

(vi) Convert the preprocessed dataframe to list of dictionaries, upload each item(dictionary) using predefined create_item() function.

In [ ]:
companies_dict = df.to_dict(orient='records')    

count = 0 # To show upload status
for item in companies_dict:
    if count%1000 == 0:
        print(count)
    container.create_item(body=item)
    count+=1


Ensure that these steps have been completed for all datasets. The following steps will assume 3 containers in the database (Companies, Research Papers, Patent Data)

### Step 4 - Query the CosmosDB containers

The SQL query below returns the company of details of companies containing OpenAlex concept Ids 2777904410 and 144133560 by quering the companies container we created on CosmosDB. Using the company IDs returned in this query, aggregated investment amounts for these companies can be fetched using a groupby query on investments container.

In [1]:
def query_items(container, company_name):
    print('\nQuerying for an  Item by Partition Key\n')
    # Including the partition key value of account_number in the WHERE filter results in a more efficient query
    items = list(container.query_items(
        query="SELECT * FROM companies WHERE ARRAY_CONTAINS(companies.openalex_concept_ids,[2777904410,144133560],True)",enable_cross_partition_query=True
    ))
    # print('Item queried by Partition Key {0}'.format(items[0].get("id")))
    return items

**Similar query can be performed for other containers to fetch academic and patent documents pertaining to specific concept IDs. We can then perform aggregations, joins and compare trends for research papers, patents and investment rounds belonging to the same concept IDs, shown in [Webapp Readme](web_application/web_application_readme.md).**